Jupyter notebook for doing the interview task for UMC:

**Analyse the drug side-effect reports (def:_narratives_)**

- look at differences between COVID and non-COVID cases
- what can we say about the text?
- suggestions in assignment:
    - first person? why?
    - redundant narrative? why?
- self investigations
    - text lengths and distributions wrt cases -- classifier. Histogram as feature for a Naive Bayes
        - Statistics
    - What features make it first person?
        - Statistics
    - redundancy detection:
        - duplication detection regexp within narrative
        - duplication of information from other columns and data files?
        - fluff content vs. entropy content
    - vocabulary differences: word frequency distributions TF-IDF 
    - content itself: keywords?
    - timing of narrative wrt symptoms?
    - look at all possibilities and their relevant narratives: FN, FP, TP, TN
    - *Sentiment analysis:* TextBlob, Vader, LLM?
    - *Topic models with LDA*


In [1]:
import pickle
import pandas as pd
from IPython.display import display
import numpy as np

data_filename = "2020VAERSDATA"
vax_filename = "2020VAERSVAX"
symptoms_filename = "2020VAERSSYMPTOMS"

Read in data and display relevant columns 

In [10]:
read_full_data_FLAG = False
# ---
if read_full_data_FLAG:
    with open(data_filename+".pickle", 'rb') as fh:
        df_data, cols_data = pickle.load(fh)
        display(df_data[cols_data].head(5))
    print("\n\n")
    with open(vax_filename+".pickle", 'rb') as fh:
        df_vax, cols_vax = pickle.load(fh)
        display(df_vax[cols_vax].head(5))
    print("\n\n")
    with open(symptoms_filename+".pickle", 'rb') as fh:
        df_symptoms, cols_symptoms = pickle.load(fh)
        display(df_symptoms[cols_symptoms].head(5))
else:
    with open("data_sample.pickle", 'rb') as fh:
        df_data, df_vax, df_symptoms = pickle.load(fh)
        cols_data = ["VAERS_ID"]
        cols_vax = ["VAERS_ID","VAX_NAME"]



Check the total number of patient entries:

In [7]:
# Data Check
print(df_data[cols_data[0]].shape)
print(df_vax[cols_data[0]].shape)
print(df_symptoms[cols_data[0]].shape)

print(df_data[cols_data[0]].nunique())
print(df_vax[cols_data[0]].nunique())
print(df_symptoms[cols_data[0]].nunique())

print(set(df_data["VAERS_ID"]).symmetric_difference(set(df_vax["VAERS_ID"])))
print(set(df_data["VAERS_ID"]).symmetric_difference(set(df_symptoms["VAERS_ID"])))

(287,)
(320,)
(346,)
287
287
287
set()
set()


In [74]:
# function
def merge_on_vaers_id(dfa, dfb):
    return pd.merge(dfa, dfb, on="VAERS_ID", how="inner")

In [8]:
# temp:: DELETE
ids = [1123, 1345, 2345, 8876]
names = ["Karaboudjan", "covid-19", "co vid", "COVID-19" ]
dict4df = {
    "VAERS_ID": ids,
    "VAERS_VAX":names
}
df_toy = pd.DataFrame(dict4df)
display(df_toy)


# get all rows where column is related to covid
query_keys = ["covid", "cov"]
df_covid = df_toy[df_toy["VAERS_VAX"].str.replace(" ", "").str.contains('|'.join(query_keys), case=False, regex=True)]
display(df_covid)

,VAERS_ID,VAERS_VAX
0,1123,Karaboudjan
1,1345,covid-19
2,2345,co vid
3,8876,COVID-19


,VAERS_ID,VAERS_VAX
1,1345,covid-19
2,2345,co vid
3,8876,COVID-19


In [20]:
# Filter vax dataframe for covid entries
df_vax_covid = df_vax[df_vax[cols_vax[1]].str.replace(" ", "").str.contains('|'.join(query_keys), case=False, regex=True)]
display(df_vax_covid.shape)
# select corresponding VAERS_ID in data and symptoms dataframes
df_data_covid = df_data[df_data["VAERS_ID"].isin(df_vax_covid["VAERS_ID"])]
df_symptoms_covid = df_symptoms[df_symptoms["VAERS_ID"].isin(df_vax_covid["VAERS_ID"])]
display(df_data_covid.shape)
print("Symptoms file has more data entries for same number of VAERS_ID: {}".format(df_symptoms_covid.shape))

# Merge vaxination and patient data on VAERS_ID
df_data_vax_covid = pd.merge(df_data_covid, df_vax_covid, on="VAERS_ID", how="inner")
display(merged_data.columns)


(46, 8)

(46, 35)

Symptoms file has more data entries for same number of VAERS_ID: (57, 11)


Index(['VAERS_ID', 'RECVDATE', 'STATE', 'AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'SEX',
       'RPT_DATE', 'SYMPTOM_TEXT', 'DIED', 'DATEDIED', 'L_THREAT', 'ER_VISIT',
       'HOSPITAL', 'HOSPDAYS', 'X_STAY', 'DISABLE', 'RECOVD', 'VAX_DATE',
       'ONSET_DATE', 'NUMDAYS', 'LAB_DATA', 'V_ADMINBY', 'V_FUNDBY',
       'OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX', 'SPLTTYPE',
       'FORM_VERS', 'TODAYS_DATE', 'BIRTH_DEFECT', 'OFC_VISIT', 'ER_ED_VISIT',
       'ALLERGIES', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES',
       'VAX_ROUTE', 'VAX_SITE', 'VAX_NAME'],
      dtype='object')

In [24]:
pd.set_option('display.max_colwidth', None)
display(merged_data[["VAERS_ID", "SYMPTOM_TEXT", "VAX_NAME"]])

,VAERS_ID,SYMPTOM_TEXT,VAX_NAME
0,902418,Patient experienced mild numbness traveling from injection site up and down arm that subsided over 20 minutes.,COVID19 (COVID19 (PFIZER-BIONTECH))
1,902440,C/O Headache,COVID19 (COVID19 (PFIZER-BIONTECH))
2,902446,"felt warm, hot and face and ears were red and flushed.",COVID19 (COVID19 (PFIZER-BIONTECH))
3,902464,within 15 minutes progressive light-headedness leading to near-syncope and diaphoresis. After 20 minutes symptoms subsided.,COVID19 (COVID19 (PFIZER-BIONTECH))
4,902465,"Pt felt wave come over body @ 1218 starting in head and going down. Bad taste in mouth, tingling in body , legs, back , across stomach, BP 150/100 P 120@ 1219, EMS activated. BP 120/80, P 80 Pt alert and oriented, Pt declined transport and Benadryl. Symptoms come and go, pt feels better but then bad taste in mouth starts, shaking of hands, tingling starts again in stomach and back. @ 1300 pt requests Benadryl, 25 mg administered. Pt notified family by phone of circumstances and family in transit.@1324 BP 120/80, P 84, tongue tingling and pt reports smelling chemical smell. @1345 Pt complained of mouth itching, EMS activated and will transport to Medical Center. Pt oriented and transported at @13",COVID19 (COVID19 (PFIZER-BIONTECH))
5,902468,"Within 1 minute, patient complained of symptoms of lightheadedness, flushing, asked for water. Symptoms persisted, reported vagal and ""spacey"", vitals were 117/91, HR 67, O2 sat 99% on room air. Reported chest heaviness, shortness of breath and within 5 minutes developed rigors and urge to defecate. 911 called, repeat vital 150/89 HR 113 O2sat 97%, continues to want to defecate. 1 loose BM, transferred to ED",COVID19 (COVID19 (PFIZER-BIONTECH))
6,902479,"rPfizer-BionNTech COVID-19 Vaccine EUA 5-7 minutes after the vaccine Associate stated she did not feel right, mentioned chest pain. ""My chest feels funny. It feels like when you have really bad heartburn coming on"". ""I feel flushed like when you get contrast for a CT"". Pulse 90 BP 160/90 checked later 130/90",COVID19 (COVID19 (PFIZER-BIONTECH))
7,902490,"Headache, body ache",COVID19 (COVID19 (PFIZER-BIONTECH))
8,902491,"Within a few minutes of receiving the COVID 19 vaccination, patient developed lightheadedness, shortness of breath, headache, and some nausea. She did get some redness to her neck and upper chest. No recent illness. Had elevated BPs ranged from 158/103 to 207/126 . HR ranged from 82-106. O2 sats always > 96%. Temp 37.1 C. Received Tylenol 1000 mg PO, Dexamethasone 10 mg IV, diphenhydramine 50 mg IV, famotidine 20 mg IV, ketorolac 30 mg IV, ondansetron 4 mg IV, and 1 L NS. Patient prescribed EpiPen and prednisone and discharged.",COVID19 (COVID19 (PFIZER-BIONTECH))
9,902492,About 25 minutes after receiving vaccine complained of dizziness and being hot and nauseated. No difficulty breathing. No chest pain. B/P was 130/90 and was monitored. It went down to 124/80 after he started feeling better. He was wearing sweater over shirt and it was warm in building. Took sweater off. Cool wet cloth applied to back of neck. States he had only had a donut and cup of hot chocolate before receiving vaccine. Sprite and peanut butter crackers given. Became nauseated after eating peanut butter crackers Blood pressure monitored monitored. He laid on exam table for about 15 minutes. He felt better. Stood up and walked to conference room for another 15 minutes. Stated he felt much better and was ready to leave. Coworker drove him back. Received email from him letting us know he had made it back and they had stopped and eaten pizza on the way. Received text from coworker that he was dizzy and seeing spots and that his blood pressure had been 120/80 and then spiked to 160/100. Coworkers taking him to ER at Hospital for evaluation.,COVID19 (COVID19 (PFIZER-BIONTECH))


In [25]:
def char_count(text):
    return len(str(text))

In [27]:
merged_data["ST_LEN"] = merged_data["SYMPTOM_TEXT"].apply(lambda x: char_count(x))
display(merged_data[["ST_LEN", "SYMPTOM_TEXT"]])

,ST_LEN,SYMPTOM_TEXT
0,110,Patient experienced mild numbness traveling from injection site up and down arm that subsided over 20 minutes.
1,12,C/O Headache
2,54,"felt warm, hot and face and ears were red and flushed."
3,124,within 15 minutes progressive light-headedness leading to near-syncope and diaphoresis. After 20 minutes symptoms subsided.
4,707,"Pt felt wave come over body @ 1218 starting in head and going down. Bad taste in mouth, tingling in body , legs, back , across stomach, BP 150/100 P 120@ 1219, EMS activated. BP 120/80, P 80 Pt alert and oriented, Pt declined transport and Benadryl. Symptoms come and go, pt feels better but then bad taste in mouth starts, shaking of hands, tingling starts again in stomach and back. @ 1300 pt requests Benadryl, 25 mg administered. Pt notified family by phone of circumstances and family in transit.@1324 BP 120/80, P 84, tongue tingling and pt reports smelling chemical smell. @1345 Pt complained of mouth itching, EMS activated and will transport to Medical Center. Pt oriented and transported at @13"
5,417,"Within 1 minute, patient complained of symptoms of lightheadedness, flushing, asked for water. Symptoms persisted, reported vagal and ""spacey"", vitals were 117/91, HR 67, O2 sat 99% on room air. Reported chest heaviness, shortness of breath and within 5 minutes developed rigors and urge to defecate. 911 called, repeat vital 150/89 HR 113 O2sat 97%, continues to want to defecate. 1 loose BM, transferred to ED"
6,311,"rPfizer-BionNTech COVID-19 Vaccine EUA 5-7 minutes after the vaccine Associate stated she did not feel right, mentioned chest pain. ""My chest feels funny. It feels like when you have really bad heartburn coming on"". ""I feel flushed like when you get contrast for a CT"". Pulse 90 BP 160/90 checked later 130/90"
7,19,"Headache, body ache"
8,541,"Within a few minutes of receiving the COVID 19 vaccination, patient developed lightheadedness, shortness of breath, headache, and some nausea. She did get some redness to her neck and upper chest. No recent illness. Had elevated BPs ranged from 158/103 to 207/126 . HR ranged from 82-106. O2 sats always > 96%. Temp 37.1 C. Received Tylenol 1000 mg PO, Dexamethasone 10 mg IV, diphenhydramine 50 mg IV, famotidine 20 mg IV, ketorolac 30 mg IV, ondansetron 4 mg IV, and 1 L NS. Patient prescribed EpiPen and prednisone and discharged."
9,1054,About 25 minutes after receiving vaccine complained of dizziness and being hot and nauseated. No difficulty breathing. No chest pain. B/P was 130/90 and was monitored. It went down to 124/80 after he started feeling better. He was wearing sweater over shirt and it was warm in building. Took sweater off. Cool wet cloth applied to back of neck. States he had only had a donut and cup of hot chocolate before receiving vaccine. Sprite and peanut butter crackers given. Became nauseated after eating peanut butter crackers Blood pressure monitored monitored. He laid on exam table for about 15 minutes. He felt better. Stood up and walked to conference room for another 15 minutes. Stated he felt much better and was ready to leave. Coworker drove him back. Received email from him letting us know he had made it back and they had stopped and eaten pizza on the way. Received text from coworker that he was dizzy and seeing spots and that his blood pressure had been 120/80 and then spiked to 160/100. Coworkers taking him to ER at Hospital for evaluation.


---
## Here forward I will just check the functionality of all the things I need to work out on one single report text.

In [47]:
symptom_text = "Pt felt wave come over body @ 1218 starting in head and going down. Bad taste in mouth, tingling in body , legs, back , across stomach, BP 150/100 P 120@ 1219, EMS activated. BP 120/80, P 80 Pt alert and oriented, Pt declined transport and Benadryl. Symptoms come and go, pt feels better but then bad taste in mouth starts, shaking of hands, tingling starts again in stomach and back. @ 1300 pt requests Benadryl, 25 mg administered. Pt notified family by phone of circumstances and family in transit.@1324 BP 120/80, P 84, tongue tingling and pt reports smelling chemical smell. @1345 Pt complained of mouth itching, EMS activated and will transport to Medical Center. Pt oriented and transported at @13"
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()

tknzr = Tokenizer(nlp.vocab)
tknzr = nlp.tokenizer

# Number of tokens
tokens = tknzr(symptom_text)
print(len(tokens))

142


In [48]:
# temp:: DELETE
import spacy
from spacy.matcher import Matcher

passive_sentences = [
    "John was accused of committing crimes by David.",
    "She was sent a cheque for a thousand euros.",
    "He was given a book for his birthday.",
    "He will be sent away to school.",
    "The meeting was called off.",
    "He was looked after by his grandmother.",
]
active_sentences = [
    "David accused John of committing crimes.",
    "Someone sent her a cheque for a thousand euros.",
    "I gave him a book for his birthday.",
    "They will send him away to school.",
    "They called off the meeting.",
    "His grandmother looked after him."
]
composite_sentences = [
    "Three men seized me, and I was carried to the car."
]


# Load spaCy pipeline (model)
nlp = spacy.load('en_core_web_sm')


In [59]:
# temp:: DELETE
# Create pattern to match passive voice use
passive_rules = [
    [{'DEP': 'nsubjpass'}, {'DEP': 'aux', 'OP': '*'}, {'DEP': 'auxpass'}, {'TAG': 'VBN'}],
    [{'DEP': 'nsubjpass'}, {'DEP': 'aux', 'OP': '*'}, {'DEP': 'auxpass'}, {'TAG': 'VBZ'}],
    [{'DEP': 'nsubjpass'}, {'DEP': 'aux', 'OP': '*'}, {'DEP': 'auxpass'}, {'TAG': 'RB'}, {'TAG': 'VBN'}],
]
# Create pattern to match active voice use
active_rules = [
    [{'DEP': 'nsubj'}, {'TAG': 'VBD', 'DEP': 'ROOT'}],
    [{'DEP': 'nsubj'}, {'TAG': 'VBP'}, {'TAG': 'VBG', 'OP': '!'}],
    [{'DEP': 'nsubj'}, {'DEP': 'aux', 'OP': '*'}, {'TAG': 'VB'}],
    [{'DEP': 'nsubj'}, {'DEP': 'aux', 'OP': '*'}, {'TAG': 'VBG'}],
    [{'DEP': 'nsubj'}, {'TAG': 'RB', 'OP': '*'}, {'TAG': 'VBG'}],
    [{'DEP': 'nsubj'}, {'TAG': 'RB', 'OP': '*'}, {'TAG': 'VBZ'}],
    [{'DEP': 'nsubj'}, {'TAG': 'RB', 'OP': '+'}, {'TAG': 'VBD'}],
]

 Parts of speech: [link](https://universaldependencies.org/docs/en/dep/)

In [60]:
# temp:: DELETE
matcher = Matcher(nlp.vocab)  # Init. the matcher with a vocab (note matcher vocab must share same vocab with docs)
matcher.add('Passive',  passive_rules)  # Add passive rules to matcher
matcher.add('Active', active_rules)  # Add active rules to matcher
text = passive_sentences + active_sentences + composite_sentences  # Combine various passive/active sentences

for sentence in text:
    doc = nlp(sentence)  # Process text with spaCy model
    matches = matcher(doc)  # Get matches
    print("-"*40 + "\n" + sentence)
    if len(matches) > 0:
        for match_id, start, end in matches:
            string_id = nlp.vocab.strings[match_id]
            span = doc[start:end]  # the matched span
            print("\t{}: {}".format(string_id, span.text))
    else:
        print("\tNo active or passive voice detected.")


----------------------------------------
John was accused of committing crimes by David.
	Passive: John was accused
----------------------------------------
She was sent a cheque for a thousand euros.
	Passive: She was sent
----------------------------------------
He was given a book for his birthday.
	Passive: He was given
----------------------------------------
He will be sent away to school.
	Passive: He will be sent
----------------------------------------
The meeting was called off.
	Passive: meeting was called
----------------------------------------
He was looked after by his grandmother.
	Passive: He was looked
----------------------------------------
David accused John of committing crimes.
	Active: David accused
----------------------------------------
Someone sent her a cheque for a thousand euros.
	Active: Someone sent
----------------------------------------
I gave him a book for his birthday.
	Active: I gave
----------------------------------------
They will send him awa

In [69]:
def tense_tally(symptom_text):
    sentences = symptom_text.split('.')
    num_sentences = len(sentences)
    actives = 0
    passives = 0
    neither = 0
    for s in sentences:
        doc = nlp(s)  # Process text with spaCy model
        matches = matcher(doc)  # Get matches
        # print("-"*40 + "\n" + s)
        if len(matches) > 0:
            for match_id, start, end in matches:
                string_id = nlp.vocab.strings[match_id]
                span = doc[start:end]  # the matched span
                # print("\t{}: {}".format(string_id, span.text))
                if string_id == "Active":
                    actives += 1
                elif string_id == "Passive":
                    passives += 1
        else:
            # print("\tNo active or passive voice detected.")
            neither += 1
    return [actives, passives, neither, actives+passives+neither, num_sentences]

In [71]:
tally = tense_tally(symptom_text)
print(tally)

merged_data["SYMPTOM_TENSE_TALLY"] = merged_data["SYMPTOM_TEXT"].apply(lambda x: tense_tally(x))

[9, 0, 2, 11, 9]


In [72]:
display(merged_data[["SYMPTOM_TENSE_TALLY", "SYMPTOM_TEXT"]])

,SYMPTOM_TENSE_TALLY,SYMPTOM_TEXT
0,"[1, 0, 1, 2, 2]",Patient experienced mild numbness traveling from injection site up and down arm that subsided over 20 minutes.
1,"[0, 0, 1, 1, 1]",C/O Headache
2,"[0, 0, 2, 2, 2]","felt warm, hot and face and ears were red and flushed."
3,"[1, 0, 2, 3, 3]",within 15 minutes progressive light-headedness leading to near-syncope and diaphoresis. After 20 minutes symptoms subsided.
4,"[9, 0, 2, 11, 9]","Pt felt wave come over body @ 1218 starting in head and going down. Bad taste in mouth, tingling in body , legs, back , across stomach, BP 150/100 P 120@ 1219, EMS activated. BP 120/80, P 80 Pt alert and oriented, Pt declined transport and Benadryl. Symptoms come and go, pt feels better but then bad taste in mouth starts, shaking of hands, tingling starts again in stomach and back. @ 1300 pt requests Benadryl, 25 mg administered. Pt notified family by phone of circumstances and family in transit.@1324 BP 120/80, P 84, tongue tingling and pt reports smelling chemical smell. @1345 Pt complained of mouth itching, EMS activated and will transport to Medical Center. Pt oriented and transported at @13"
5,"[2, 0, 3, 5, 5]","Within 1 minute, patient complained of symptoms of lightheadedness, flushing, asked for water. Symptoms persisted, reported vagal and ""spacey"", vitals were 117/91, HR 67, O2 sat 99% on room air. Reported chest heaviness, shortness of breath and within 5 minutes developed rigors and urge to defecate. 911 called, repeat vital 150/89 HR 113 O2sat 97%, continues to want to defecate. 1 loose BM, transferred to ED"
6,"[6, 0, 2, 8, 5]","rPfizer-BionNTech COVID-19 Vaccine EUA 5-7 minutes after the vaccine Associate stated she did not feel right, mentioned chest pain. ""My chest feels funny. It feels like when you have really bad heartburn coming on"". ""I feel flushed like when you get contrast for a CT"". Pulse 90 BP 160/90 checked later 130/90"
7,"[0, 0, 1, 1, 1]","Headache, body ache"
8,"[4, 0, 7, 11, 11]","Within a few minutes of receiving the COVID 19 vaccination, patient developed lightheadedness, shortness of breath, headache, and some nausea. She did get some redness to her neck and upper chest. No recent illness. Had elevated BPs ranged from 158/103 to 207/126 . HR ranged from 82-106. O2 sats always > 96%. Temp 37.1 C. Received Tylenol 1000 mg PO, Dexamethasone 10 mg IV, diphenhydramine 50 mg IV, famotidine 20 mg IV, ketorolac 30 mg IV, ondansetron 4 mg IV, and 1 L NS. Patient prescribed EpiPen and prednisone and discharged."
9,"[9, 0, 11, 20, 20]",About 25 minutes after receiving vaccine complained of dizziness and being hot and nauseated. No difficulty breathing. No chest pain. B/P was 130/90 and was monitored. It went down to 124/80 after he started feeling better. He was wearing sweater over shirt and it was warm in building. Took sweater off. Cool wet cloth applied to back of neck. States he had only had a donut and cup of hot chocolate before receiving vaccine. Sprite and peanut butter crackers given. Became nauseated after eating peanut butter crackers Blood pressure monitored monitored. He laid on exam table for about 15 minutes. He felt better. Stood up and walked to conference room for another 15 minutes. Stated he felt much better and was ready to leave. Coworker drove him back. Received email from him letting us know he had made it back and they had stopped and eaten pizza on the way. Received text from coworker that he was dizzy and seeing spots and that his blood pressure had been 120/80 and then spiked to 160/100. Coworkers taking him to ER at Hospital for evaluation.


In [78]:
merged_df_sample = merge_on_vaers_id(df_data, df_vax)
display(merged_df_sample.shape)

# Select data that are not covid
merged_df_sample_non_covid = merged_df_sample[~merged_df_sample[cols_vax[1]].str.replace(" ", "").str.contains('|'.join(query_keys), case=False, regex=True)]
display(merged_df_sample_non_covid.shape)

(320, 42)

(274, 42)

In [79]:
merged_df_sample_non_covid["SYMPTOM_TENSE_TALLY"] = merged_df_sample_non_covid["SYMPTOM_TEXT"].apply(lambda x: tense_tally(x))

/tmp/ipykernel_12346/5316417.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_sample_non_covid["SYMPTOM_TENSE_TALLY"] = merged_df_sample_non_covid["SYMPTOM_TEXT"].apply(lambda x: tense_tally(x))


In [80]:
display(merged_data[["SYMPTOM_TENSE_TALLY", "SYMPTOM_TEXT"]])

,SYMPTOM_TENSE_TALLY,SYMPTOM_TEXT
0,"[1, 0, 1, 2, 2]",Patient experienced mild numbness traveling from injection site up and down arm that subsided over 20 minutes.
1,"[0, 0, 1, 1, 1]",C/O Headache
2,"[0, 0, 2, 2, 2]","felt warm, hot and face and ears were red and flushed."
3,"[1, 0, 2, 3, 3]",within 15 minutes progressive light-headedness leading to near-syncope and diaphoresis. After 20 minutes symptoms subsided.
4,"[9, 0, 2, 11, 9]","Pt felt wave come over body @ 1218 starting in head and going down. Bad taste in mouth, tingling in body , legs, back , across stomach, BP 150/100 P 120@ 1219, EMS activated. BP 120/80, P 80 Pt alert and oriented, Pt declined transport and Benadryl. Symptoms come and go, pt feels better but then bad taste in mouth starts, shaking of hands, tingling starts again in stomach and back. @ 1300 pt requests Benadryl, 25 mg administered. Pt notified family by phone of circumstances and family in transit.@1324 BP 120/80, P 84, tongue tingling and pt reports smelling chemical smell. @1345 Pt complained of mouth itching, EMS activated and will transport to Medical Center. Pt oriented and transported at @13"
5,"[2, 0, 3, 5, 5]","Within 1 minute, patient complained of symptoms of lightheadedness, flushing, asked for water. Symptoms persisted, reported vagal and ""spacey"", vitals were 117/91, HR 67, O2 sat 99% on room air. Reported chest heaviness, shortness of breath and within 5 minutes developed rigors and urge to defecate. 911 called, repeat vital 150/89 HR 113 O2sat 97%, continues to want to defecate. 1 loose BM, transferred to ED"
6,"[6, 0, 2, 8, 5]","rPfizer-BionNTech COVID-19 Vaccine EUA 5-7 minutes after the vaccine Associate stated she did not feel right, mentioned chest pain. ""My chest feels funny. It feels like when you have really bad heartburn coming on"". ""I feel flushed like when you get contrast for a CT"". Pulse 90 BP 160/90 checked later 130/90"
7,"[0, 0, 1, 1, 1]","Headache, body ache"
8,"[4, 0, 7, 11, 11]","Within a few minutes of receiving the COVID 19 vaccination, patient developed lightheadedness, shortness of breath, headache, and some nausea. She did get some redness to her neck and upper chest. No recent illness. Had elevated BPs ranged from 158/103 to 207/126 . HR ranged from 82-106. O2 sats always > 96%. Temp 37.1 C. Received Tylenol 1000 mg PO, Dexamethasone 10 mg IV, diphenhydramine 50 mg IV, famotidine 20 mg IV, ketorolac 30 mg IV, ondansetron 4 mg IV, and 1 L NS. Patient prescribed EpiPen and prednisone and discharged."
9,"[9, 0, 11, 20, 20]",About 25 minutes after receiving vaccine complained of dizziness and being hot and nauseated. No difficulty breathing. No chest pain. B/P was 130/90 and was monitored. It went down to 124/80 after he started feeling better. He was wearing sweater over shirt and it was warm in building. Took sweater off. Cool wet cloth applied to back of neck. States he had only had a donut and cup of hot chocolate before receiving vaccine. Sprite and peanut butter crackers given. Became nauseated after eating peanut butter crackers Blood pressure monitored monitored. He laid on exam table for about 15 minutes. He felt better. Stood up and walked to conference room for another 15 minutes. Stated he felt much better and was ready to leave. Coworker drove him back. Received email from him letting us know he had made it back and they had stopped and eaten pizza on the way. Received text from coworker that he was dizzy and seeing spots and that his blood pressure had been 120/80 and then spiked to 160/100. Coworkers taking him to ER at Hospital for evaluation.
